# Practical ResNet

### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

### Data

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 30
batch_size = 100
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='./data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data/', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified


### Model

In [3]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        identity = x.clone()
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        """
        Args:
            block (ResidualBlock): ResidualBlock class
            layers (list): [2, 2, 2]
            num_classes (int, optional): Defaults to 10.
        """
        super().__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 64, layers[2], stride=2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
    
    def make_layer(self, block, out_channels, blocks, stride):
        downsample = None
        if self.in_channels != out_channels or stride != 1:
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        residual_blocks = []
        residual_blocks.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            residual_blocks.append(block(self.in_channels, out_channels))
        return nn.Sequential(*residual_blocks)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

### Training

In [7]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
decay = 0
for epoch in range(num_epochs):
    # Decay learning rate by a factor of 0.5 every 20 epochs
    if (epoch + 1) % 20 == 0:
        decay += 1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5 ** decay)
        print(f"The learning rate is now {optimizer.param_groups[0]['lr']}")
    # Train
    correct = 0
    iterations = 0
    iter_loss = 0.0
    model.train() # Set model to training mode (for dropout and batchnorm)
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        iter_loss += loss.item()
        iterations += 1
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_loss = iter_loss / iterations
    train_accuracy = 100 * correct / len(train_dataset)
    
    # Test
    correct = 0
    iterations = 0
    iter_loss = 0.0
    model.eval() # Set model to evaluation mode (for dropout and batchnorm)
    for i, (images, labels) in enumerate(test_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        # Calculate loss
        loss = criterion(outputs, labels)
        iter_loss += loss.item()
        iterations += 1
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
    
    test_loss = iter_loss / iterations
    test_accuracy = 100 * correct / len(test_dataset)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}, Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}")

Epoch 1/30, Train Loss: 1.3963, Train Acc: 48.59, Test Loss: 1.0691, Test Acc: 61.33
Epoch 2/30, Train Loss: 0.9910, Train Acc: 64.77, Test Loss: 1.0918, Test Acc: 60.90
Epoch 3/30, Train Loss: 0.8317, Train Acc: 70.87, Test Loss: 1.1946, Test Acc: 59.78
Epoch 4/30, Train Loss: 0.7294, Train Acc: 74.61, Test Loss: 0.8460, Test Acc: 70.64
Epoch 5/30, Train Loss: 0.6616, Train Acc: 76.96, Test Loss: 0.8404, Test Acc: 71.82
Epoch 6/30, Train Loss: 0.6136, Train Acc: 78.70, Test Loss: 0.8420, Test Acc: 72.93
Epoch 7/30, Train Loss: 0.5787, Train Acc: 79.99, Test Loss: 0.6759, Test Acc: 77.32
Epoch 8/30, Train Loss: 0.5438, Train Acc: 81.25, Test Loss: 0.5977, Test Acc: 79.65
Epoch 9/30, Train Loss: 0.5097, Train Acc: 82.52, Test Loss: 0.6285, Test Acc: 78.86
Epoch 10/30, Train Loss: 0.4946, Train Acc: 83.00, Test Loss: 0.5913, Test Acc: 79.42
Epoch 11/30, Train Loss: 0.4706, Train Acc: 83.73, Test Loss: 0.6241, Test Acc: 79.72
Epoch 12/30, Train Loss: 0.4505, Train Acc: 84.55, Test Loss: 0